# Scrittopoli - Giornata 1

In [2]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import mylib.spreadsheet as ms
import mylib.elo as melo
import mylib.utility as mu


In [3]:
day="20171214_142354" # definitivo
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day

formazioni_csv="data/formazioni_giornata1.csv"

calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])




## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [4]:
quadro=calendario.loc[1].copy()

riposano=quadro[quadro["squadra 2"]=="(riposo)"][["squadra 1"]]
riposano



,,squadra 1
girone,partita,
A,2,Gli ultimi


In [5]:
quadro=quadro[quadro["squadra 2"]!="(riposo)"]
quadro["vs"]="contro"

quadro=quadro[["squadra 1","vs","squadra 2"]]

quadro=quadro.sort_index()
quadro

squadra 1      vs           squadra 2
girone partita                                                           
A      1                          D Euforiche  contro        La threesome
B      1                          Green House  contro          I Bananari
       2        Gli scrittori del quartierino  contro      the 3 BrOthers
C      1                        Living Stones  contro  Retroguardia Trash
       2                          Le TreMende  contro  Armata Brancaleone

In [6]:
importlib.reload(mu)
mu.to_table_html_multi(quadro)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>La threesome</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>I Bananari</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>the 3 BrOthers</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Retroguardia Trash</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Le TreMende</td>
<td style='text

## Fase 1. Dalle formazioni ai match

In [7]:
squadre=scrittopoli.copy()
squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

# Versione random di prova, da sostituire con lettura della formazione via csv o spreadsheet
data=[]
for sq in squadre.index.unique():
    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
    data.append([sq,capitano,riserva]+titolari)
formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])
formazioni=formazioni.set_index("squadra")

#formazioni=pd.read_csv(formazioni_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()

In [8]:

formazioni

,capitano,riserva,titolare 1,titolare 2
squadra,,,,
Green House,Plata,Bango Skank,Joyopi,Plata
La threesome,Komorebi,Luna,Komorebi,mina99
I Bananari,simone volponi,Niko,simone volponi,massimopud
Armata Brancaleone,Skorpiòs,Ella F.,AdStr,Skorpiòs
Living Stones,Vincenzo Iennaco,Macleo,Vincenzo Iennaco,Ljuset
Retroguardia Trash,AlexComan,AlexComan,Eudes,Andrea28
Le TreMende,Thea,Emy,sefora,Thea
Gli scrittori del quartierino,camparino,Federico72,camparino,Marty12
the 3 BrOthers,AryaSophia,ITG,AryaSophia,ettore70


In [75]:
def parametri_giornata(giornata,calendario,elo):
    rank=elo.copy().reset_index().set_index("giocatore")["rank %d" % (giornata-1)]
    cal=calendario[calendario["squadra 2"]!="(riposo)"].loc[1]
    return cal,rank

def sort_by_rank(rank,a,b):
    if rank[a]>rank[b]: return [a,b]
    if rank[a]<rank[b]: return [b,a]
    L=[a,b]
    random.shuffle(L)
    return L

def calcola_accoppiamenti(giornata,formazioni,calendario,elo_squadre):
    cal,rank=parametri_giornata(1,calendario,elo_squadre)
    data=[]
    for girone,partita in cal.index:
        for n in [1,2]:
            squadra=cal.loc[(girone,partita)]["squadra %d" % n]
            riserva=formazioni.loc[squadra]["riserva"]
            capitano=formazioni.loc[squadra]["capitano"]
            titolare_1=formazioni.loc[squadra]["titolare 1"]
            titolare_2=formazioni.loc[squadra]["titolare 2"]
            data.append([girone,partita,squadra,capitano,riserva]+sort_by_rank(rank,titolare_1,
                                                                               titolare_2))    
    X=pd.DataFrame(data,
                   columns=["girone","partita","squadra","capitano","riserva",
                            "match 1","match 2"])
    X=X.set_index(["girone","partita"]).sort_index()
    return X

accoppiamenti=calcola_accoppiamenti(1,formazioni,calendario,elo_squadre)
accoppiamenti


squadra     capitano         riserva  \
girone partita                                                    
A      1               Green House  Bango Skank           Plata   
       1                I Bananari         Niko  simone volponi   
B      1             Living Stones       Macleo          Macleo   
       1              La threesome       mina99          mina99   
C      1               D Euforiche         Kuno          queffe   
       1        Retroguardia Trash        Eudes           Eudes   
       2            the 3 BrOthers   AryaSophia      AryaSophia   
       2        Armata Brancaleone     Skorpiòs           AdStr   

                         match 1    match 2  
girone partita                               
A      1             Bango Skank     Joyopi  
       1              massimopud       Niko  
B      1        Vincenzo Iennaco     Ljuset  
       1                Komorebi       Luna  
C      1                Marcello       Kuno  
       1                Andrea28  AlexComan  
       2                     ITG   ettore70  
       2                 Ella F.   Skorpiòs

## Fase 2. Creazione del tabellone per voti e controlli

In [287]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

old_data=s.get_giornata(1)
s.set_giornata(1,accoppiamenti,gironi,old_data)


## Fase 3. Recupero dati

In [342]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
voti=s.get_giornata(1)


In [343]:
### Simulazione di riempimento del tabellone

import string


def random_title(x):
    size=random.randint(10,20)
    chars=string.ascii_lowercase +string.ascii_uppercase + string.digits
    chars+="               "
    S=''.join(random.choice(chars) for _ in range(size))
    S=S.strip()
    S=" ".join(S.split())
    return S

def random_len(x):
    return random.randint(300,8200)

def random_riserva(x):
    if random.choice([True,False]):
        return ""
    return "12-08 23:23"

def random_verifica(x):
    if random.choice([True,False,True,True,True,True,True,True,True,True,True]):
        return "x"
    return ""


voti["base","titolo"]=voti["base","titolo"].apply(random_title)
voti["base","ingresso riserva"]=voti["base","ingresso riserva"].apply(random_riserva)
voti["verifiche","lunghezza"]=voti["verifiche","lunghezza"].apply(random_len)
voti["verifiche","link"]=voti["verifiche","link"].apply(random_verifica)
voti["verifiche","link commento"]=voti["verifiche","link commento"].apply(random_verifica)
voti["verifiche","tag"]=voti["verifiche","tag"].apply(random_verifica)
voti["verifiche","commento valido"]=voti["verifiche","commento valido"].apply(random_verifica)

for G,altri in [ ("A",[("B",1),("C",1),("C",2)]),
                 ("B",[("A",1),("C",1),("C",2)]),
                 ("C",[("A",1),("B",1)]) ]:
    for sq in gironi[gironi[G]!="(riposo)"][G]:
        for gir,part in altri:
            for m in ["match 1","match 2"]:
                giocatore_1=accoppiamenti.loc[gir,part][m].iloc[0]
                giocatore_2=accoppiamenti.loc[gir,part][m].iloc[1]
                goal_1=random.randint(0,2)
                goal_2=2-goal_1
                voti.set_value(giocatore_1,(G,sq),goal_1)
                voti.set_value(giocatore_2,(G,sq),goal_2)

def calc_penalita_lungh(x):
    return -int(x>8000)

def calc_penalita(x):
    if x: return 0
    return -1

voti["verifiche","penalità"]=voti["verifiche","lunghezza"].apply(calc_penalita_lungh)
for k in ["tag","link","link commento","commento valido"]:
    voti["verifiche","penalità"]+=voti["verifiche",k].apply(calc_penalita)


In [344]:
voti["base","goal giocatore"]=voti[["A","B","C"]].sum(axis=1,level=1).sum(axis=1)+voti["verifiche","penalità"]
voti

base                       verifiche           \
                 ingresso riserva                titolo lunghezza tag link   
titolare                                                                     
Bango Skank           12-08 23:23     oZHDWmL fOr S q9g      1832   x    x   
Joyopi                12-08 23:23           e nsk5Lhf9w      2055   x    x   
massimopud            12-08 23:23       VB Tz rfkOOWH M      7289   x    x   
Niko                  12-08 23:23    4O N etF LWd9Yje R      3815   x    x   
Vincenzo Iennaco                    cMT9d4 0o MEQYr0O1d      2880   x    x   
Ljuset                12-08 23:23  vz5 9U Xtmc rGY DDML      3087   x    x   
Komorebi              12-08 23:23             WSsflg jc      7793   x    x   
Luna                                 WhcsfB4Lg WPE6 E 0      3929   x    x   
Marcello              12-08 23:23          p jG mFOrTEY      5809   x    x   
Kuno                  12-08 23:23        6M 7vG xaYX JV      3374   x    x   
Andrea28                            FMTgiXhqA cNndfbEV5      7119   x    x   
AlexComan                          Cd 3U7PO Bvwns5l7uEu      1871   x    x   
ITG                   12-08 23:23   2W7g64C8bDuFjpS4DUD       822   x    x   
ettore70                              UTM D EB Je56nQ 0      6712   x    x   
Ella F.               12-08 23:23              z9lk r n      7497   x    x   
Skorpiòs              12-08 23:23    8NrgSaeMhq 42U6WAm      7949   x    x   

                                                                  A  \
                 link commento commento valido penalità Green House   
titolare                                                              
Bango Skank                  x               x        0           0   
Joyopi                       x               x        0           0   
massimopud                   x               x        0           0   
Niko                         x               x        0           0   
Vincenzo Iennaco             x               x        0           0   
Ljuset                       x               x        0           2   
Komorebi                     x               x        0           2   
Luna                         x               x        0           0   
Marcello                     x               x        0           2   
Kuno                         x               x        0           2   
Andrea28                     x               x        0           0   
AlexComan                    x               x        0           0   
ITG                          x               x        0           0   
ettore70                                     x       -1           1   
Ella F.                      x               x        0           2   
Skorpiòs                     x               x        0           1   

                                                                      B  \
                 I Bananari Gli scrittori del quartierino Living Stones   
titolare                                                                  
Bango Skank               0                             0             2   
Joyopi                    0                             0             2   
massimopud                0                             0             0   
Niko                      0                             0             0   
Vincenzo Iennaco          1                             2             0   
Ljuset                    2                             0             0   
Komorebi                  1                             0             0   
Luna                      0                             2             0   
Marcello                  2                             2             1   
Kuno                      1                             2             2   
Andrea28                  0                             0             1   
AlexComan                 1                             0             0   
ITG                       0                             2             0

In [357]:
accoppiamenti

risultati=accoppiamenti.copy()

risultati=risultati.reset_index().set_index("squadra")
risultati["match 1 goal"]=0
risultati["match 1 riserva"]=False
risultati["match 2 goal"]=0
risultati["match 2 riserva"]=False
risultati["goal"]=0
risultati["punteggio"]=0

for sq in risultati.index:
    m1=risultati.loc[sq]["match 1"]
    m2=risultati.loc[sq]["match 2"]
    goal_1=voti.loc[m1]["base","goal giocatore"]
    goal_2=voti.loc[m2]["base","goal giocatore"]
    risultati.set_value(sq,"match 1 goal",goal_1)
    risultati.set_value(sq,"match 2 goal",goal_2)
    risultati.set_value(sq,"goal",goal_1+goal_2)
    ris_1=bool(voti.loc[m1]["base","ingresso riserva"])
    risultati.set_value(sq,"match 1 riserva",ris_1)
    if ris_1: continue
    risultati.set_value(sq,"match 2 riserva",bool(voti.loc[m2]["base","ingresso riserva"]))
    
risultati=risultati.reset_index().set_index(["girone","partita"])
sc_dati=[]
for girone,partita in risultati.index.unique():
    squadra_1=risultati.loc[girone,partita]["squadra"].iloc[0]
    squadra_2=risultati.loc[girone,partita]["squadra"].iloc[1]
    goal_1=risultati.loc[girone,partita]["goal"].iloc[0]
    goal_2=risultati.loc[girone,partita]["goal"].iloc[1]
    sc_dati.append([girone,partita,squadra_1,squadra_2,goal_1,goal_2])
    
sc_dati

risultati=risultati.reset_index().set_index(["girone","partita","squadra"])

for girone,partita,squadra_1,squadra_2,goal_1,goal_2 in sc_dati:
    if goal_1>goal_2:
        p1=3
        p2=0
    elif goal_1<goal_2:
        p1=0
        p2=3
    else:
        p1=1
        p2=1
    risultati.set_value( (girone,partita,squadra_1),"punteggio",p1)
    risultati.set_value( (girone,partita,squadra_2),"punteggio",p2)
        
risultati


capitano         riserva  \
girone partita squadra                                           
A      1       Green House         Bango Skank           Plata   
               I Bananari                 Niko  simone volponi   
B      1       Living Stones            Macleo          Macleo   
               La threesome             mina99          mina99   
C      1       D Euforiche                Kuno          queffe   
               Retroguardia Trash        Eudes           Eudes   
       2       the 3 BrOthers       AryaSophia      AryaSophia   
               Armata Brancaleone     Skorpiòs           AdStr   

                                            match 1    match 2  match 1 goal  \
girone partita squadra                                                         
A      1       Green House              Bango Skank     Joyopi             4   
               I Bananari                massimopud       Niko            10   
B      1       Living Stones       Vincenzo Iennaco     Ljuset             7   
               La threesome                Komorebi       Luna             7   
C      1       D Euforiche                 Marcello       Kuno             8   
               Retroguardia Trash          Andrea28  AlexComan             4   
       2       the 3 BrOthers                   ITG   ettore70             5   
               Armata Brancaleone           Ella F.   Skorpiòs             7   

                                   match 1 riserva  match 2 goal  \
girone partita squadra                                             
A      1       Green House                    True             7   
               I Bananari                     True             7   
B      1       Living Stones                 False            10   
               La threesome                   True             4   
C      1       D Euforiche                    True            10   
               Retroguardia Trash            False             2   
       2       the 3 BrOthers                 True             5   
               Armata Brancaleone             True             6   

                                   match 2 riserva  goal  punteggio  
girone partita squadra                                               
A      1       Green House                   False    11          0  
               I Bananari                    False    17          3  
B      1       Living Stones                  True    17          3  
               La threesome                  False    11          0  
C      1       D Euforiche                   False    18          3  
               Retroguardia Trash            False     6          0  
       2       the 3 BrOthers                False    10          0  
               Armata Brancaleone            False    13          3

In [372]:
sched_data=[]
print('<table style="margin:auto;">')
print('<tr><th>girone</th><th>partita</th><th>match 1</th><th>match 2</th><th>risultato</th></tr>')
for girone,partita,squadra_1,squadra_2,goal_1,goal_2 in sc_dati:
    s1_goal_m1=risultati.loc[(girone,partita,squadra_1)]["match 1 goal"]
    s1_goal_m2=risultati.loc[(girone,partita,squadra_1)]["match 2 goal"]
    s2_goal_m1=risultati.loc[(girone,partita,squadra_2)]["match 1 goal"]
    s2_goal_m2=risultati.loc[(girone,partita,squadra_2)]["match 2 goal"]
    pdata=(girone,squadra_1,squadra_2,s1_goal_m1,s2_goal_m1,s1_goal_m2,s2_goal_m2,goal_1,goal_2)
    sched_data.append(pdata)
    print('<tr><td>%s</td><td>%s - %s</td><td>%d-%d</td><td>%d-%d</td><td>%d-%d</td></tr>' % pdata)

print('</table>')


<table style="margin:auto;">
<tr><th>girone</th><th>partita</th><th>match 1</th><th>match 2</th><th>risultato</th></tr>
<tr><td>A</td><td>Green House - I Bananari</td><td>4-10</td><td>7-7</td><td>11-17</td></tr>
<tr><td>B</td><td>Living Stones - La threesome</td><td>7-7</td><td>10-4</td><td>17-11</td></tr>
<tr><td>C</td><td>D Euforiche - Retroguardia Trash</td><td>8-4</td><td>10-2</td><td>18-6</td></tr>
<tr><td>C</td><td>the 3 BrOthers - Armata Brancaleone</td><td>5-7</td><td>5-6</td><td>10-13</td></tr>
</table>


In [376]:
t=[("partita","girone"),
   ("partita","squadra 1"),
   ("partita","squadra 2"),
   ("match 1","squadra 1"),
   ("match 1","squadra 2"),
   ("match 2","squadra 1"),
   ("match 2","squadra 2"),
   ("totale","squadra 1"),
   ("totale","squadra 2"),]
tindex=pd.MultiIndex.from_tuples(t)

partite=pd.DataFrame(sched_data,columns=tindex)
partite=partite.set_index(("partita","girone"))
partite.index.names=["girone"]
partite

partita                       match 1             match 2  \
             squadra 1           squadra 2 squadra 1 squadra 2 squadra 1   
girone                                                                     
A          Green House          I Bananari         4        10         7   
B        Living Stones        La threesome         7         7        10   
C          D Euforiche  Retroguardia Trash         8         4        10   
C       the 3 BrOthers  Armata Brancaleone         5         7         5   

                    totale            
       squadra 2 squadra 1 squadra 2  
girone                                
A              7        11        17  
B              4        17        11  
C              2        18         6  
C              6        10        13

In [384]:
cl_dati=[]

for girone in gironi.columns:
    for sq in gironi[gironi[girone]!="(riposo)"][girone]:
        cl_dati.append([girone,sq])

classifica=pd.DataFrame(cl_dati,columns=["girone","squadra"])
classifica["totale"]=0

## QUI merge da sistemare
t_ris=risultati.reset_index().set_index(["girone","squadra"])[["punteggio"]]
t_ris.merge(classifica,left_index=True,index_right=True)

TypeError: merge() got an unexpected keyword argument 'index_right'

In [367]:
## classifica e ranking


In [ ]:
### salvataggi
## CSV
# risultati
# voti
# partite
# classifica
# rank

## spreadsheet
# classifica
# rank
# partite
